In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('../Housing/Housing.csv')

In [3]:
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
# Describing the data
df.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


Price: Ranges from 1,750,000 to 13,300,000 with an average of approximately 4,766,729.

Area: Varies from 1,650 to 16,200 square units, with an average area of about 5,150.54.

Bedrooms: Range from 1 to 6 bedrooms.

Bathrooms: The number of bathrooms ranges from 1 to 4.

Stories: Properties have between 1 and 4 stories.

Parking: The number of parking spaces ranges from 0 to 3.

In [5]:
df.isnull().sum()

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64

In [6]:
df.nunique()

price               219
area                284
bedrooms              6
bathrooms             4
stories               4
mainroad              2
guestroom             2
basement              2
hotwaterheating       2
airconditioning       2
parking               4
prefarea              2
furnishingstatus      3
dtype: int64

Price: 219 unique values.

Area: 284 unique values.

Bedrooms: 6 unique values.

Bathrooms: 4 unique values.

Stories: 4 unique values.

Parking: 4 unique values.

Other categorical columns like mainroad, guestroom, basement, hotwaterheating, airconditioning, prefarea, and furnishingstatus have 2 or 3 unique values each.

In [7]:
X = df.drop('price', axis=1)
y = df['price']

In [8]:
X.head()

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [9]:
# Identifying categorical and numerical columns
categorical_cols = [col for col in X.columns if X[col].dtype == "object"]
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

In [10]:
categorical_cols

['mainroad',
 'guestroom',
 'basement',
 'hotwaterheating',
 'airconditioning',
 'prefarea',
 'furnishingstatus']

In [11]:
numerical_cols

['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

In [12]:
# Create the LabelEncoder instance
label_encoder = LabelEncoder()

In [13]:
# Apply the encoder to the selected columns
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

In [14]:
X.head()

,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,7420,4,2,3,1,0,0,0,1,2,1,0
1,8960,4,4,4,1,0,0,0,1,3,0,0
2,9960,3,2,2,1,0,1,0,0,2,1,1
3,7500,4,2,2,1,0,1,0,1,3,1,0
4,7420,4,1,2,1,1,1,0,1,2,0,0


In [15]:
y.head()

0    13300000
1    12250000
2    12250000
3    12215000
4    11410000
Name: price, dtype: int64

In [16]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [17]:
model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 10)

In [18]:
model.fit(X_train, y_train)

XGBRegressor(alpha=10, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, ...)

In [20]:
area = input("Enter Area")
bedrooms = input("Enter no of Bedrooms")
bathrooms = input("Enter no of Bathrooms")
stories = input("Enter no of Stories")
mainroad = input("Does it have mainroad? (Enter 1 if Yes or 0 if No)")
guestroom = input("Does it have guestroom? (Enter 1 if Yes or 0 if No)")
basement = input("Does it have basement? (Enter 1 if Yes or 0 if No)")
hotwaterheating = input("Does it have hotwaterheating? (Enter 1 if Yes or 0 if No)")
airconditioning = input("Does it have airconditioning? (Enter 1 if Yes or 0 if No)")
parking = input("Enter number of parking slots")
prefarea = input("Does it have prefarea? (Enter 1 if Yes or 0 if No)")
furnishingstatus = input("Is it furnished? (Enter 0 if furnished, 1 if semi-furnished, 2 if unfurnished)")

Enter Area5456
Enter no of Bedrooms2
Enter no of Bathrooms2
Enter no of Stories1
Does it have mainroad? (Enter 1 if Yes or 0 if No)0
Does it have guestroom? (Enter 1 if Yes or 0 if No)1
Does it have basement? (Enter 1 if Yes or 0 if No)0
Does it have hotwaterheating? (Enter 1 if Yes or 0 if No)1
Does it have airconditioning? (Enter 1 if Yes or 0 if No)0
Enter number of parking slots1
Does it have prefarea? (Enter 1 if Yes or 0 if No)0
Is it furnished? (Enter 0 if furnished, 1 if semi-furnished, 2 if unfurnished)1


In [21]:
new_data = [int(area), int(bedrooms), int(bathrooms), int(stories), int(mainroad), int(guestroom), int(basement), int(hotwaterheating), int(airconditioning), int(parking), int(prefarea), int(furnishingstatus)]

In [22]:
new_data

[5456, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 1]

In [23]:
new_data_array = np.array(new_data).reshape(1, -1)
new_data_array

array([[5456,    2,    2,    1,    0,    1,    0,    1,    0,    1,    0,
           1]])

In [24]:
predicted_price = model.predict(new_data_array)

In [25]:
predicted_price[0]

4854406.0